## Construct Wealth Index with PCA

In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
DHS_SURVEY_DATA = 'D:/Satellite/data/weath index with gps and date(1).csv'

# column order: continuous variables, binary categorial variables, multiple categorial variables
CONT_COLUMNS = ['room.number']
BI_CAT_COLUMNS = ['Electricity', 'telephone', 'radio', 'television', 'car', 'motorcycle', 'refrigerator']
MUL_CAT_COLUMNS = ['floor.meterial', 'water.source', 'toilt.type']
PCA_COLUMNS = ['room.per_person', 'Electricity', 'telephone', 'radio', 'television', 'car', 'motorcycle', 'refrigerator', 'floor.meterial', 'water.source', 'toilt.type']

SA_5_CC = ['IA', 'PK', 'BD', 'NP', 'LK']
SA_5_COUNTRIES = ['India', 'Pakistan', 'Bangladesh', 'Nepal', 'Sri Lanka']  # {"IA", "PK", "BD", "NP", "LK"}

MIN_YEAR = 2000

In [5]:
# load DSH data
dhs_df = pd.read_csv(DHS_SURVEY_DATA, float_precision='high', index_col=False).reset_index(drop=True)
dhs_df.rename(columns={'Unnamed: 0': 'svyid'}, inplace=True)
print(dhs_df.shape)

# Exclude data before MIN_YEAR
dhs_df = dhs_df.loc[dhs_df['survey.year'] >= MIN_YEAR]
print(dhs_df.shape)
dhs_df.head()

C:\Users\xiaon\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3170: DtypeWarning: Columns (3,29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2282337, 44)
(2121176, 44)


,svyid,source,id,HHID,household.number,drinkingwater.source,water.source,toilt.type,Electricity,radio,...,lon,lat,URBAN_RURA,LATNUM,LONGNUM,ALT_DEM,ALT_GPS,survey.date,survey.month,survey.year
869,27138,BDHR41DT.ZIP,870,55 3,6,21.0,21.0,31.0,0.0,0.0,...,89.514095,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000
870,27139,BDHR41DT.ZIP,871,55 10,2,21.0,21.0,21.0,0.0,0.0,...,89.514095,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000
871,27140,BDHR41DT.ZIP,872,55 17,3,21.0,21.0,31.0,0.0,0.0,...,89.514095,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000
872,27141,BDHR41DT.ZIP,873,55 24,1,21.0,21.0,31.0,0.0,0.0,...,89.514095,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000
873,27142,BDHR41DT.ZIP,874,55 31,3,21.0,21.0,31.0,0.0,0.0,...,89.514095,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000


In [6]:
# Recode multi-category variables and process missing/NaN cells
def encode_floor(value):
    x = value // 10
    if x < 1 or x > 3:
        return np.nan
    else:
        return x * 1.0

def encode_water(value):
    x = value // 10
    if x < 1 or x > 5:
        return np.nan
    else:
        return 6.0 - x

def encode_toilet(value):
    x = value // 10
    if x < 1 or x > 3:
        return np.nan
    else:
        return 4.0 - x

dhs_df['floor.meterial'] = dhs_df['floor.meterial'].map(encode_floor)
dhs_df['water.source'] = dhs_df['water.source'].map(encode_water)
dhs_df['toilt.type'] = dhs_df['toilt.type'].map(encode_toilet)
for col in BI_CAT_COLUMNS:
    dhs_df[col] = dhs_df[col].replace(9.0, np.nan)
for col in CONT_COLUMNS:
    dhs_df[col] = dhs_df[col].replace(99.0, np.nan)

dhs_df['room.per_person'] = dhs_df['room.number'] / dhs_df['household.number']

dhs_df.head()

,svyid,source,id,HHID,household.number,drinkingwater.source,water.source,toilt.type,Electricity,radio,...,lat,URBAN_RURA,LATNUM,LONGNUM,ALT_DEM,ALT_GPS,survey.date,survey.month,survey.year,room.per_person
869,27138,BDHR41DT.ZIP,870,55 3,6,21.0,4.0,1.0,0.0,0.0,...,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN
870,27139,BDHR41DT.ZIP,871,55 10,2,21.0,4.0,2.0,0.0,0.0,...,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN
871,27140,BDHR41DT.ZIP,872,55 17,3,21.0,4.0,1.0,0.0,0.0,...,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN
872,27141,BDHR41DT.ZIP,873,55 24,1,21.0,4.0,1.0,0.0,0.0,...,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN
873,27142,BDHR41DT.ZIP,874,55 31,3,21.0,4.0,1.0,0.0,0.0,...,25.322485,R,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN


In [7]:
# Prepare ownership asset viriables to calculate wealth index
feature_df_all = dhs_df.loc[:, ['svyid', 'cc'] + PCA_COLUMNS].reset_index(drop=True)
print(feature_df_all.shape)
feature_df_all.head()

(2121176, 13)


,svyid,cc,room.per_person,Electricity,telephone,radio,television,car,motorcycle,refrigerator,floor.meterial,water.source,toilt.type
0,27138,BD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,4.0,1.0
1,27139,BD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,4.0,2.0
2,27140,BD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,4.0,1.0
3,27141,BD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,4.0,1.0
4,27142,BD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,1.0,4.0,1.0


In [8]:
def cal_wealth_index(feature_df, cc_list=None):
    # drop any rows with NaN values
    feature_df_nonna = feature_df.dropna(axis=0, how='any')
    if cc_list:
        feature_df_nonna = feature_df_nonna.loc[feature_df_nonna['cc'].isin(cc_list)]
    print("Shape of non-NaN features:", feature_df_nonna.shape)
    
    # PCA (only keep the first principal component)
    pca = PCA(n_components=1, whiten=True)
    wealth_index_pooled = pca.fit_transform(feature_df_nonna.iloc[:, 2:])
    print('variance_ratio:', pca.explained_variance_ratio_)
    print('singular_values:', pca.singular_values_)
    print('mean:', np.mean(wealth_index_pooled))
    print('std:', np.std(wealth_index_pooled))
    
    # Multivariate feature impuation
    imp = IterativeImputer(max_iter=20, random_state=0, verbose=True)
    features_all = imp.fit_transform(feature_df.iloc[:, 2:])
    wealth_index_pooled = pca.transform(features_all)
    print('mean:', np.mean(wealth_index_pooled))
    print('std:', np.std(wealth_index_pooled))
    
    return wealth_index_pooled

In [10]:
# Calculate wealth index with PCA and multivariate feature impuation
print('==== wealthpooled ====')
dhs_df['wealthpooled'] = cal_wealth_index(feature_df_all, cc_list=None)
print('\n==== wealthpooled5country ====')
dhs_df['wealthpooled5country'] = cal_wealth_index(feature_df_all, cc_list=SA_5_CC)
print(dhs_df.shape)
dhs_df.head()

==== wealthpooled ====
Shape of non-NaN features: (302780, 13)
variance_ratio: [0.51162814]
singular_values: [826.71242417]
mean: 5.557055281986648e-17
std: 0.9999983486346095
[IterativeImputer] Completing matrix with shape (2121176, 11)
[IterativeImputer] Change: 2.7745691794969676, scaled tolerance: 0.023 
[IterativeImputer] Change: 2.8206453061350465, scaled tolerance: 0.023 
[IterativeImputer] Change: 1.3570040784704411, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.4899482942191021, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.21584948989390235, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.16669334852569118, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.1285321820123132, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.09905022683132725, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.0763217931865367, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.058807864504371976, scaled tolerance: 0.023 
[IterativeImputer] Change: 0.04531291

,svyid,source,id,HHID,household.number,drinkingwater.source,water.source,toilt.type,Electricity,radio,...,LATNUM,LONGNUM,ALT_DEM,ALT_GPS,survey.date,survey.month,survey.year,room.per_person,wealthpooled,wealthpooled5country
869,27138,BDHR41DT.ZIP,870,55 3,6,21.0,4.0,1.0,0.0,0.0,...,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN,0.854688,-1.013478
870,27139,BDHR41DT.ZIP,871,55 10,2,21.0,4.0,2.0,0.0,0.0,...,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN,0.592385,-0.559065
871,27140,BDHR41DT.ZIP,872,55 17,3,21.0,4.0,1.0,0.0,0.0,...,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN,0.854688,-1.013478
872,27141,BDHR41DT.ZIP,873,55 24,1,21.0,4.0,1.0,0.0,0.0,...,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN,0.854688,-1.013478
873,27142,BDHR41DT.ZIP,874,55 31,3,21.0,4.0,1.0,0.0,0.0,...,25.322485,89.514095,24.0,9999.0,1201,1,2000,NaN,0.854688,-1.013478


## Households to Clusters

In [11]:
import sys
from collections import defaultdict

In [12]:
CC_DATA = 'D:/Satellite/data/cc.csv'
DHS_CLUTERS_DATA = f'../data/dhs_clusters_from_{MIN_YEAR}.csv'
DHS_WEALTH_INDEX_DATA = f'../data/dhs_wealth_index_from_{MIN_YEAR}.csv'

In [13]:
dhs_df.columns.values

array(['svyid', 'source', 'id', 'HHID', 'household.number',
       'drinkingwater.source', 'water.source', 'toilt.type',
       'Electricity', 'radio', 'television', 'refrigerator', 'bicycle',
       'motorcycle', 'car', 'floor.meterial', 'walls.meterial',
       'roof.material', 'room.number', 'telephone', 'shared.toilet',
       'stove', 'ventilation', 'cooked.in.house', 'separate.kitchen',
       'smoke.frequency', 'wealthindex.urban', 'wealthindex.factor', 'cc',
       'phase', 'dbs', 'mothersurvey.month', 'mothersurvey.year',
       'mothersurvey.day', 'lon', 'lat', 'URBAN_RURA', 'LATNUM',
       'LONGNUM', 'ALT_DEM', 'ALT_GPS', 'survey.date', 'survey.month',
       'survey.year', 'room.per_person', 'wealthpooled',
       'wealthpooled5country'], dtype=object)

In [14]:
# Grouping and aggregating

# each cluster has unique tuple (lon, lat, source)
# columns: ['lon', 'lat', 'source', 'cc', 'dbs', 'survey.year', URBAN_RURA', 'wealthpooled', 'wealthpooled5country']
grouped = dhs_df.groupby([dhs_df['lon'], dhs_df['lat'], dhs_df['source']])
dhs_cluster_df_cc = grouped['cc'].agg(lambda x: x.value_counts().index[0]).reset_index()
dhs_cluster_df_year = grouped['survey.year'].agg(lambda x: x.value_counts().index[0]).reset_index()
dhs_cluster_df_urban = grouped['URBAN_RURA'].agg(lambda x: x.value_counts().index[0]).reset_index()
dhs_cluster_df_dbs = grouped['dbs'].agg(lambda x: x.value_counts().index[0]).reset_index()
dhs_cluster_df_wealth = grouped['wealthpooled'].mean().reset_index()
dhs_cluster_df_wealth_5_cc = grouped['wealthpooled5country'].mean().reset_index()
dhs_cluster_df_households = grouped.size().reset_index()
dhs_cluster_df_households.rename(columns={0: 'households'}, inplace=True)

### dhs_clusters.csv

In [15]:
group_key = ['lon', 'lat', 'source']
dhs_cluster_df = dhs_cluster_df_cc.merge(
    dhs_cluster_df_year, on=group_key).merge(
    dhs_cluster_df_wealth, on=group_key).merge(
    dhs_cluster_df_wealth_5_cc, on=group_key).merge(
    dhs_cluster_df_households, on=group_key).merge(
    dhs_cluster_df_urban, on=group_key)

# Map cc to country name
cc_df = pd.read_csv(CC_DATA, float_precision='high', index_col=False)
cc_to_country = dict(zip(list(cc_df['Code']), list(cc_df['Country Name'])))
dhs_cluster_df['cc'] = dhs_cluster_df['cc'].map(lambda x: cc_to_country[x])

# convert URBAN_RURA column from string to int: 'U' (urban) => 1, 'R' (rural) => 0
urban_rural_str_to_int = {'U': 1, 'R': 0}
dhs_cluster_df['URBAN_RURA'] = dhs_cluster_df['URBAN_RURA'].map(lambda x: urban_rural_str_to_int[x])

dhs_cluster_df.rename(columns={'cc': 'country'}, inplace=True)
dhs_cluster_df.rename(columns={'survey.year': 'year'}, inplace=True)
dhs_cluster_df.rename(columns={'URBAN_RURA': 'urban_rural'}, inplace=True)

dhs_cluster_df.sort_values(by=['country', 'year'], axis=0, ascending=True, inplace=True)
dhs_cluster_df

,lon,lat,source,country,year,wealthpooled,wealthpooled5country,households,urban_rural
17369,19.417422,42.045868,ALHR51DT.ZIP,Albania,2008,-0.213189,1.243182,19,0
17446,19.483347,40.735403,ALHR51DT.ZIP,Albania,2008,-0.728684,1.555386,18,0
17466,19.490580,40.678990,ALHR51DT.ZIP,Albania,2008,-1.143543,1.814193,18,0
17583,19.560772,41.899937,ALHR51DT.ZIP,Albania,2008,-1.065338,1.785889,18,0
17586,19.562051,40.807737,ALHR51DT.ZIP,Albania,2008,-1.125585,1.831241,19,0
...,...,...,...,...,...,...,...,...,...
29663,32.850670,-16.966440,ZWHR71DT.ZIP,Zimbabwe,2015,-0.284547,0.873594,28,1
29709,32.875480,-19.847010,ZWHR71DT.ZIP,Zimbabwe,2015,0.709170,-0.506523,26,0
29739,32.885590,-18.441580,ZWHR71DT.ZIP,Zimbabwe,2015,0.286499,0.210907,28,0
29752,32.890810,-18.174680,ZWHR71DT.ZIP,Zimbabwe,2015,0.594996,-0.247505,27,0


In [16]:
dhs_cluster_df.to_csv(DHS_CLUTERS_DATA, index=False)

### dhs_wealth_index.csv

In [17]:
dhs_index_df = dhs_cluster_df_cc.merge(
    dhs_cluster_df_year, on=group_key).merge(
    dhs_cluster_df_dbs, on=group_key).merge(
    dhs_cluster_df_wealth, on=group_key).merge(
    dhs_cluster_df_wealth_5_cc, on=group_key).merge(
    dhs_cluster_df_households, on=group_key).merge(
    dhs_cluster_df_urban, on=group_key)

# Map cc to country name
dhs_index_df['country'] = dhs_index_df['cc'].map(lambda x: cc_to_country[x])

# Map [cc][year] to svyid
dhs_index_df['svyid'] = dhs_index_df[["cc", "survey.year"]].apply(lambda x: f'{x["cc"]}{x["survey.year"]}', axis=1)

dhs_index_df.rename(columns={'lon': 'LONGNUM'}, inplace=True)
dhs_index_df.rename(columns={'lat': 'LATNUM'}, inplace=True)
dhs_index_df.rename(columns={'cc': 'cname'}, inplace=True)
dhs_index_df.rename(columns={'survey.year': 'year'}, inplace=True)
dhs_index_df.rename(columns={'dbs': 'region'}, inplace=True)

dhs_index_df.sort_values(by=['country', 'year'], axis=0, ascending=True, inplace=True)

# Add column "cluster" to index clusters for each svyid
def add_cluster_index(x):
    x['cluster'] = x.reset_index().index + 1
    return x
grouped = dhs_index_df.groupby([dhs_index_df['svyid']])
dhs_index_df_cluster_idx = grouped.apply(add_cluster_index).reset_index()[['cluster']]
dhs_index_df = dhs_index_df_cluster_idx.join(dhs_index_df.reset_index())

dhs_index_df = dhs_index_df[['cluster', 'svyid', 'wealthpooled', 'wealthpooled5country', 'year', 'cname', 'country', 
                             'region', 'households', 'LATNUM', 'LONGNUM', 'URBAN_RURA', 'source']]
dhs_index_df

,cluster,svyid,wealthpooled,wealthpooled5country,year,cname,country,region,households,LATNUM,LONGNUM,URBAN_RURA,source
0,1,AL2008,-0.213189,1.243182,2008,AL,Albania,other,19,42.045868,19.417422,R,ALHR51DT.ZIP
1,2,AL2008,-0.728684,1.555386,2008,AL,Albania,other,18,40.735403,19.483347,R,ALHR51DT.ZIP
2,3,AL2008,-1.143543,1.814193,2008,AL,Albania,other,18,40.678990,19.490580,R,ALHR51DT.ZIP
3,4,AL2008,-1.065338,1.785889,2008,AL,Albania,other,18,41.899937,19.560772,R,ALHR51DT.ZIP
4,5,AL2008,-1.125585,1.831241,2008,AL,Albania,other,19,40.807737,19.562051,R,ALHR51DT.ZIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78686,396,ZW2015,-0.284547,0.873594,2015,ZW,Zimbabwe,africa,28,-16.966440,32.850670,U,ZWHR71DT.ZIP
78687,397,ZW2015,0.709170,-0.506523,2015,ZW,Zimbabwe,africa,26,-19.847010,32.875480,R,ZWHR71DT.ZIP
78688,398,ZW2015,0.286499,0.210907,2015,ZW,Zimbabwe,africa,28,-18.441580,32.885590,R,ZWHR71DT.ZIP
78689,399,ZW2015,0.594996,-0.247505,2015,ZW,Zimbabwe,africa,27,-18.174680,32.890810,R,ZWHR71DT.ZIP


In [18]:
dhs_index_df.to_csv(DHS_WEALTH_INDEX_DATA, index=False)

## Analysis

In [19]:
dhs_cluster_df.country.value_counts()

India                        28390
Egypt                         5263
Peru                          4217
Kenya                         2598
Bangladesh                    2102
Malawi                        2075
Nigeria                       1808
Dominican Republic            1784
Cambodia                      1689
Uganda                        1585
Pakistan                      1516
Philippines                   1373
Senegal                       1224
Ethiopia                      1194
Zimbabwe                      1189
Madagascar                    1184
Lesotho                       1176
Albania                       1154
Haiti                         1144
Jordan                        1100
Tanzania                      1074
Zambia                        1040
Indonesia                     1014
Bolivia                        994
Angola                         969
Timor-Leste                    911
Nepal                          894
Mali                           825
Namibia             

In [20]:
for country in SA_5_COUNTRIES:
    print(f'{country}:', dhs_cluster_df.loc[(dhs_cluster_df['country'] == country)].year.unique())

India: [2015 2016]
Pakistan: [2006 2007 2017 2018]
Bangladesh: [2000 2004 2007 2011 2014]
Nepal: [2001 2006 2016 2017]
Sri Lanka: []


## South Asia Data

In [22]:
DHS_CLUTERS_DATA_DEMO = f'../data/South_Asia_5_countries/dhs_clusters_South_Asia_from_{MIN_YEAR}.csv'

In [23]:
dhs_cluster_sa_df = dhs_cluster_df.loc[dhs_cluster_df['country'].isin(SA_5_COUNTRIES)]
dhs_cluster_sa_df

,lon,lat,source,country,year,wealthpooled,wealthpooled5country,households,urban_rural
66369,88.094460,24.727418,BDHR41DT.ZIP,Bangladesh,2000,0.474337,-0.373352,29,0
66480,88.232915,24.925880,BDHR41DT.ZIP,Bangladesh,2000,0.237621,0.035174,33,0
66549,88.274134,24.500782,BDHR41DT.ZIP,Bangladesh,2000,0.691554,-0.722465,30,0
66564,88.286215,24.589249,BDHR41DT.ZIP,Bangladesh,2000,-0.496278,1.077311,17,1
66810,88.497805,24.780564,BDHR41DT.ZIP,Bangladesh,2000,1.194342,-1.159412,25,0
...,...,...,...,...,...,...,...,...,...
46604,75.589857,35.564494,PKHR71DT.ZIP,Pakistan,2018,-0.524518,1.116176,26,0
46727,75.655105,35.492510,PKHR71DT.ZIP,Pakistan,2018,-0.497181,1.097792,27,0
47545,75.962071,35.133919,PKHR71DT.ZIP,Pakistan,2018,-0.952856,1.660920,26,0
47754,76.070929,34.881647,PKHR71DT.ZIP,Pakistan,2018,-0.318725,0.836875,26,0


In [24]:
dhs_cluster_sa_df.to_csv(DHS_CLUTERS_DATA_DEMO, index=False)